In [4]:
import cv2
import numpy as np
import os
from PIL import Image
import dlib

hog_face_detector = dlib.get_frontal_face_detector()

recognizer = cv2.face.LBPHFaceRecognizer_create()

Face_ID = -1
prev_person_name = ""
y_ID = []
x_train = []

Face_Images = os.path.join(os.getcwd(), "Face_Images")
print(Face_Images)

for root, dirs, files in os.walk(Face_Images):
    for file in files:
        if file.endswith("jpeg") or file.endswith("jpg") or file.endswith("png"):
            path = os.path.join(root, file)
            person_name = os.path.basename(root)
            print(path, person_name)

        if prev_person_name != person_name:
            Face_ID += 1
            prev_person_name = person_name

        img = cv2.imread(path)
        gray_iamge = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        faces = hog_face_detector(gray_iamge, 1)

        print(Face_ID, faces)

        for face_detection in faces:
            x, y, w, h = (
                face_detection.left(),
                face_detection.top(),
                face_detection.right(),
                face_detection.bottom(),
            )
            roi = gray_iamge[y : y + h, x : x + w]
            x_train.append(roi)
            y_ID.append(Face_ID)

            recognizer.train(x_train, np.array(y_ID))
            recognizer.save("face-trainner.yml")

c:\Users\main\Desktop\vision_project\ch03\Face_Images
c:\Users\main\Desktop\vision_project\ch03\Face_Images\Cha\1.jpg Cha
0 rectangles[[(167, 192) (390, 415)]]
c:\Users\main\Desktop\vision_project\ch03\Face_Images\Cha\2.jpg Cha
0 rectangles[[(260, 260) (527, 528)]]
c:\Users\main\Desktop\vision_project\ch03\Face_Images\Cha\3.jpg Cha
0 rectangles[[(134, 241) (455, 562)]]
c:\Users\main\Desktop\vision_project\ch03\Face_Images\Cha\4.jpg Cha
0 rectangles[[(510, 118) (696, 304)]]
c:\Users\main\Desktop\vision_project\ch03\Face_Images\Cha\5.jpg Cha
0 rectangles[[(455, 241) (776, 562)]]
c:\Users\main\Desktop\vision_project\ch03\Face_Images\Cha\cha1.jpg Cha
0 rectangles[[(290, 290) (675, 675)]]
c:\Users\main\Desktop\vision_project\ch03\Face_Images\Cha\cha2.jpg Cha
0 rectangles[[(22, 142) (290, 409)]]
c:\Users\main\Desktop\vision_project\ch03\Face_Images\Cha\cha3.jpg Cha
0 rectangles[[(170, 206) (491, 527)]]
c:\Users\main\Desktop\vision_project\ch03\Face_Images\Cha\cha4.jpg Cha
0 rectangles[[(262,

In [5]:
import cv2
import numpy as np
import os
from PIL import Image
import dlib

labels = ["Cha", "Ma"]

hog_face_detector = dlib.get_frontal_face_detector()

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("face-trainner.yml")

image_list = []

test_images = os.path.join(os.getcwd(), "test_list")

for root, dirs, files in os.walk(test_images):
    for file in files:
        if file.endswith("jpeg") or file.endswith("jpg") or file.endswith("png"):
            image_path = os.path.join(test_images, file)
            print(image_path)
            image_list.append(cv2.imread(image_path))

for img in image_list:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = hog_face_detector(gray, 1)

    for face_detection in faces:
        x, y, w, h = (
            face_detection.left(),
            face_detection.top(),
            face_detection.right(),
            face_detection.bottom(),
        )
        roi_gray = gray[y : y + h, x : x + w]

        id, conf = recognizer.predict(roi_gray)
        print(labels[id], conf)

        if conf >= 50:
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id]
            cv2.putText(img, name, (x, y), font, 1, (255, 0, 0), 2)
            cv2.rectangle(img, (x, y), (w + x, y + h), (255, 0, 0), 2)

    cv2.imshow("Preview", img)
    if cv2.waitKey() >= 0:
        continue

cv2.destroyAllWindows()

c:\Users\main\Desktop\vision_project\ch03\test_list\cha_test1.jpg
c:\Users\main\Desktop\vision_project\ch03\test_list\cha_test2.jpg
c:\Users\main\Desktop\vision_project\ch03\test_list\ma_test1.jpg
c:\Users\main\Desktop\vision_project\ch03\test_list\ma_test2.jpg
Ma 35.272522334733736
Cha 29.081021258753676
Ma 54.23262775908261
Cha 49.33385712582005


In [1]:
import cv2
import time

labels = ["Cha", "Ma"]
webcam = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("face-trainner.yml")

if not webcam.isOpened():
    print("찾을수 없어요 웹캠을")
    exit()

while webcam.isOpened():
    status, frame = webcam.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    start = time.time()
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)
    end = time.time()

    for x, y, w, h in faces:
        roi_gray = gray[y : y + h, x : x + w]
        id, conf = recognizer.predict(roi_gray)
        print(labels[id], conf)

        if conf >= 50:
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id]
            cv2.putText(frame, name, (x, y), font, 1, (255, 0, 0), 2)
            org = (50, 100)
            text = f"{end - start:.5f} seconds"
            cv2.putText(frame, text, org, font, 1, (255, 0, 0), 2)
            cv2.rectangle(frame, (x, y), (w + x, y + h), (255, 0, 0), 2)

    if status:
        cv2.imshow("test", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
webcam.release()
cv2.destroyAllWindows()

Ma 70.94801607892038
Ma 69.21335827026432
Ma 71.99278065412638
Cha 123.38216417775826
Cha 123.6770958014966
Cha 123.01484587854904
Cha 124.74869406352406
Cha 122.46365935824062
Cha 122.77592098402314
Ma 124.46480817385485
Cha 125.66952677010576
Cha 124.00200508949312
Cha 121.44813241406351
Cha 124.69009091053692
Cha 124.47891994174846
Cha 124.6954963969513
Cha 123.95316318234049
Ma 122.5234352985823
Cha 125.15486962199056
Cha 125.49851031260378
Cha 123.17195133517725
Cha 177.29127360157548
Cha 169.20675423655865
Ma 168.2818824559601
Ma 165.42221721882638
Ma 165.44034637478893
Ma 166.225437640863
Cha 168.4915346724089
Ma 167.1425307778063
Ma 163.72662081537936
Cha 158.75159368780302
Ma 156.35993688848214
Ma 156.35993688848214
Ma 153.87502768701617
Ma 159.3857761612432
Cha 161.20772878495512
Ma 159.7413011289763
Ma 153.90962879709957
Ma 154.00249596918673
Ma 153.78350516918562
Ma 158.16977237775185
Ma 159.89243706036348
Cha 158.51615490956272
Ma 157.22640751730316
Ma 154.82749700303307
C